# Chemical-Disease Relation (CDR) Tutorial

In this example, we'll be writing an application to extract *mentions of* **chemical-induced-disease relationships** from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  This tutorial will show off some of the more advanced features of Snorkel, so we'll assume you've followed the Intro tutorial.

Let's start by reloading from the last notebook.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from snorkel import SnorkelSession

session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

train = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).all()
dev = session.query(ChemicalDisease).filter(ChemicalDisease.split == 1).all()
test = session.query(ChemicalDisease).filter(ChemicalDisease.split == 2).all()

print('Training set:\t{0} candidates'.format(len(train)))
print('Dev set:\t{0} candidates'.format(len(dev)))
print('Test set:\t{0} candidates'.format(len(test)))

Training set:	21312 candidates
Dev set:	2486 candidates
Test set:	11975 candidates


# Part V: Training an LSTM extraction model

In the intro tutorial, we automatically featurized the candidates and trained a linear model over these features. Here, we'll train a more complicated model for relation extraction: an LSTM network. You can read more about LSTMs [here](https://en.wikipedia.org/wiki/Long_short-term_memory) or [here](http://colah.github.io/posts/2015-08-Understanding-LSTMs/). An LSTM is a type of recurrent neural network and automatically generates a numerical representation for the candidate based on the sentence text, so no need for featurizing explicitly as in the intro tutorial. LSTMs take longer to train, and Snorkel doesn't currently support hyperparameter searches for them. We'll train a single model here, but feel free to try out other parameter sets. Just make sure to use the development set - and not the test set - for model selection.

**Note: Again, training for more epochs than below will greatly improve performance- try it out!**

In [3]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, split=0)

In [4]:
from snorkel.annotations import load_gold_labels
from load_external_annotations import load_external_labels
load_external_labels(session, ChemicalDisease, split=1, annotator='gold')
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)

AnnotatorLabels created: 0


In [5]:
from snorkel.learning.pytorch import LSTM
train_kwargs = {
    'lr':              0.001,
    'embedding_dim':   100,
    'hidden_dim':      100,
    'n_epochs':        40,
    'dropout':         0.2,
    'rebalance':       False,
    'print_freq':      40,
    'seed':            1701,
    'batch_size':      80
}

lstm = LSTM(n_threads=1)
lstm.train(train, train_marginals, X_dev=dev, Y_dev=L_gold_dev, **train_kwargs)



[LSTM] Training model
[LSTM] n_train=19621  #epochs=40  batch size=80
[LSTM] Epoch 1 (63.96s)	Average loss=0.689270	Dev F1=36.78
[LSTM] Epoch 40 (2301.70s)	Average loss=0.662494	Dev F1=49.66
[LSTM] Model saved as <LSTM>
[LSTM] Training done (2304.41s)
[LSTM] Loaded model <LSTM>


### Scoring on the test set

Finally, we'll evaluate our performance on the blind test set of 500 documents. We'll load labels similar to how we did for the development set, and use the `score` function of our extraction model to see how we did. In addition, we create a `metrics` dictionary where we store the metrics of our model for each sentence length, where the keys are the number of sentences the relation spans (0 is reserved for the cumulative metrics).

In [6]:
from load_external_annotations import load_external_labels
load_external_labels(session, ChemicalDisease, split=2, annotator='gold')
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
lstm.score(test, L_gold_test)
tp, fp, tn, fn = lstm.error_analysis(session, test, L_gold_test, set_unlabeled_as_neg=False)
metrics = {}
sent_lengths = set()
metrics[0] = [0 for _ in range(4)]

for i, metric in enumerate([tp, fp, tn, fn]):
    for sample in metric:
        metrics[0][i] += 1 
        cand_length = int(abs(sample.chemical.sentence.position - sample.disease.sentence.position)+1)
        if cand_length in sent_lengths:
            metrics[cand_length][i] += 1 
        else:
            metrics[cand_length] = [0 for _ in range(4)]
            metrics[cand_length][i] += 1 
            sent_lengths.add(cand_length)
print(metrics)

AnnotatorLabels created: 0
Scores (Un-adjusted)
Pos. class accuracy: 0.651
Neg. class accuracy: 0.577
Precision            0.395
Recall               0.651
F1                   0.491
----------------------------------------
TP: 2321 | FP: 3562 | TN: 4849 | FN: 1243

{0: [2321, 3562, 4849, 1243], 1: [1184, 1647, 1529, 328], 2: [1137, 1915, 3320, 915]}
